In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.agents import initialize_agent, Tool
from langchain_community.embeddings import DashScopeEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
# 初始化模型


api_key = os.getenv("api_key")
base_url = os.getenv("base_url")
model = os.getenv("model")
llm = ChatOpenAI(model=model, api_key=api_key,base_url=base_url,streaming=True)

memory = ConversationBufferMemory()

# 加载外部文档
loader = TextLoader("example_doc.txt")
documents = loader.load()
embeddings = DashScopeEmbeddings(
    model="text-embedding-v3",
    dashscope_api_key=api_key,
    # other params...
)
vector_store = FAISS.from_documents(documents, embeddings)

# 定义工具
def search_docs(query):
    return vector_store.similarity_search(query)[0].page_content

tools = [Tool(name="DocumentSearch", func=search_docs, description="搜索文档")]

# 创建代理
agent = initialize_agent(tools, llm, agent_type="zero-shot-react-description", memory=memory)

# 运行
response = agent.invoke("2025年的技术趋势是什么？")
print(response)


{'input': '2025年的技术趋势是什么？', 'history': '', 'output': '2025年的技术趋势将围绕以下几个关键领域展开：  \n\n1. **人工智能与自动化**：AI将在医疗诊断、客户服务、制造业中广泛应用，推动流程自动化。Gartner预测，到2025年，70%的企业将采用AI驱动的自动化优化运营。  \n\n2. **量子计算商业化**：量子计算将在药物研发、加密和优化问题（如物流）中实现初步商业化应用，解决传统计算机难以处理的复杂问题。  \n\n3. **绿色能源与可持续技术**：可再生能源（如太阳能、风能）成本持续下降，储能技术（如固态电池）进步，碳捕捉技术商业化加速。国际能源署预计，到2025年，可再生能源将占新增发电量的80%以上。  \n\n4. **边缘计算与5G/6G融合**：低延迟网络与边缘计算结合，推动物联网设备实时数据处理，支持智慧城市、工业4.0和自动驾驶的发展。  \n\n5. **扩展现实（XR）与元宇宙**：AR/VR技术将深化在教育、远程办公和医疗培训中的应用，元宇宙场景进一步融入日常生活与商业。  \n\n6. **数字孪生技术**：通过虚拟模型优化物理系统（如工厂、城市），提升效率和可持续性。  \n\n7. **自动驾驶技术**：L4级自动驾驶可能在限定区域（如物流、公共交通）实现商业化部署，推动交通系统智能化。  \n\n8. **区块链与供应链透明化**：区块链技术将广泛应用于供应链管理，提升透明度和安全性，尤其在食品和医药行业。  \n\n9. **个性化医疗与基因编辑**：CRISPR等技术推动精准医疗发展，治疗更多遗传性疾病，如癌症和罕见病。  \n\n10. **可持续材料与循环经济**：生物降解材料和资源回收技术进步，助力实现零排放目标，企业将更关注循环经济模式。  \n\n这些趋势反映了技术与可持续发展、人机协作及数字化转型的深度融合，为未来社会和经济变革奠定基础。'}
